# 微调中文和日文的双语摘要任务模型

确保我们的模型不会过度拟合单一语言

### 预处理

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset('csv',data_files='./data/amazon_reviews_multi/train.csv')
raw_dataset = raw_dataset.rename_column('Unnamed: 0','id')
raw_chinese_dataset = raw_dataset.filter(lambda x:x['language']=='zh')
raw_japanese_dataset = raw_dataset.filter(lambda x:x['language']=='ja')

split_chinese_dataset = raw_chinese_dataset['train'].train_test_split(train_size=0.9)
test_and_valid = split_chinese_dataset['test'].train_test_split(train_size=0.5)
chinese_dataset = split_chinese_dataset
chinese_dataset['validation'] = test_and_valid.pop('train')
chinese_dataset['test'] = test_and_valid.pop('test')

split_japanese_dataset = raw_japanese_dataset['train'].train_test_split(train_size=0.9)
test_and_valid = split_japanese_dataset['test'].train_test_split(train_size=0.5)
japanese_dataset = split_japanese_dataset
japanese_dataset['validation'] = test_and_valid.pop('train')
japanese_dataset['test'] = test_and_valid.pop('test')
print(chinese_dataset)
print(japanese_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 180000
    })
    test: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 10000
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 180000
    })
    test: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 10000
    })
    

In [ ]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


show_samples(chinese_dataset)
show_samples(japanese_dataset)

chinese_dataset.set_format("pandas")
ch_df = chinese_dataset["train"][:]
# 显示前 20 个产品的数量
ch_df["product_category"].value_counts()[:20]


japanese_dataset.set_format("pandas")
ja_df = japanese_dataset["train"][:]
# 显示前 20 个产品的数量
ja_df["product_category"].value_counts()[:20]


'>> Title: 不要相信这家店，'
'>> Review: 收到货打开一看，里面简直就是垃圾，没有吊牌衣领那纸被老鼠蛟一样，里面到处都是纸碎，外面哎！！！当初我都不知怎会收货的，我现在看着它一肚子气👎'

'>> Title: 太可恶了'
'>> Review: 我已经下单快三天了，现在还没发货，我就在北京，总是显示即将发货，现在想取消订单都不行了，这是课本，我有急用的，怎么能这样！！！！'

'>> Title: 好鞋，但不适合南方夏秋季穿'
'>> Review: 很好的鞋型和舒适性。但鞋里用料为海绵，直接把富乐绅品牌档次拉低。里料头层牛皮不奢求，但二层牛皮或者猪皮，可能会容易接受。尤其广东地区，疯马皮厚实，再加上海绵鞋里，不适合穿。可惜了！'

'>> Title: シリーズ物は手強い。'
'>> Review: シリーズ化され本を、書店では売り切れで購入がかなわなかったが、アマゾンで見つかったので安堵した。入手に苦労した本である。'

'>> Title: 手軽に使える'
'>> Review: 小さな土間うちの目地の縦のラインを工具箱から出してさっと手軽に使えました。レーザー設置するほどでもないときに即使える手軽さ、そして安価。もちろん屋外で本格的に使えるものではありませんが（☆ﾏｲﾅｽ１はコレだけ）、場所を選べばとても使い勝手の良い商品でした。'

'>> Title: ゴミ'
'>> Review: 2、3回回しただけで金属の根元からもげました。笑えないです'


product_category
home                16828
wireless            14705
sports              14133
pc                  13472
kitchen             11989
automotive          11793
drugstore            9206
electronics          8909
shoes                8049
toy                  7732
beauty               7649
apparel              7363
home_improvement     6035
grocery              5774
other                5283
book                 5001
pet_products         4357
camera               3987
office_product       3852
video_games          2294
Name: count, dtype: int64

In [ ]:
def filter_books(example):
    return (
        example["product_category"] == "book"
        or example["product_category"] == "digital_ebook_purchase"
    )
chinese_dataset.reset_format()
chinese_dataset.reset_format()
japanese_dataset.reset_format()

ch_books = chinese_dataset.filter(filter_books)
ja_books = japanese_dataset.filter(filter_books)
show_samples(ch_books)

Filter:   0%|          | 0/180000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/180000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]


'>> Title: 不错'
'>> Review: 物流很给力，两天就到了。大概看了一下，有步骤讲解我觉得还不错。给满分！'

'>> Title: 错别字太多了'
'>> Review: 错别字太多了，有些地方简直都无法原谅。开始我还会标出来进行报告，后来实在是来不及。'

'>> Title: 亚马逊太垃圾了'
'>> Review: 6月28日同时买的一本书，现在还不发货，居然还没投诉电话，呸'


In [ ]:
# 将中文和日文的评论作为单个DatasetDict对象组合起来
#  Datasets 提供了一个方便的 concatenate_datasets() 函数
from datasets import concatenate_datasets,DatasetDict

books_dataset = DatasetDict()

for split in ch_books.keys():
    books_dataset[split] = concatenate_datasets(
        [ch_books[split],ja_books[split]]
    )
    books_dataset[split] = books_dataset[split].shuffle(seed=100)

show_samples(books_dataset)


'>> Title: 亏了'
'>> Review: 内容很好，但是不明白亚马逊为什么要单独把书单独买。'

'>> Title: 非常没用的一本书'
'>> Review: 其实就是讲均线多头排列，都是马后炮的描述，并没有实质性教人怎么选出这类股。'

'>> Title: 小提琴考级书正品'
'>> Review: 是正品，纸张好，碟片音效不错，下次考级书还来这里买！'


In [ ]:
books_dataset = books_dataset.filter(lambda x:x['review_title'] is not None)
books_dataset = books_dataset.map(lambda x:{"title_length":len(x['review_title'])})
books_dataset = books_dataset.filter(lambda x:x['title_length']>2)

Filter:   0%|          | 0/81016 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4413 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4444 [00:00<?, ? examples/s]

Map:   0%|          | 0/81015 [00:00<?, ? examples/s]

Map:   0%|          | 0/4413 [00:00<?, ? examples/s]

Map:   0%|          | 0/4444 [00:00<?, ? examples/s]

Filter:   0%|          | 0/81015 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4413 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4444 [00:00<?, ? examples/s]

In [ ]:
books_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'title_length'],
        num_rows: 69968
    })
    test: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'title_length'],
        num_rows: 3817
    })
    validation: Dataset({
        features: ['id', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'title_length'],
        num_rows: 3845
    })
})

### 模型

这是一种基于 T5 的有趣架构，在文本到文本任务中进行了预训练。在 T5 中，每个 NLP 任务都是以任务前缀（如 summarize: ）的形式定义的，模型根据不同的任务生成不同的文本

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\Anaconda\envs\d2lc\lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
inputs = tokenizer("我喜欢读饥饿游戏")
print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))

['▁', '我', '喜欢', '读', '饥', '饿', '游戏', '</s>']


In [ ]:
# 评论和标题的最大长度 因为使用T5模型，评论和标题要一起给，可能会超长
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["review_title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = books_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/69968 [00:00<?, ? examples/s]

Map:   0%|          | 0/3817 [00:00<?, ? examples/s]

Map:   0%|          | 0/3845 [00:00<?, ? examples/s]

### 评估

一种方法是计算两段摘要的重叠单词的数量

ROUGE算法

召回率= 重叠词数量/参考摘要的总词数  召回率越高越好
 
精确度= 重叠词数量/生成摘要总次数

In [ ]:
# !pip install rouge_score
# !pip install rouge_chinese
# !pip install jieba

from rouge_chinese import Rouge
import jieba # you can use any other word cutting library

generated_summary = ' '.join(jieba.cut("我特别特别喜欢读饥饿游戏这本书"))
reference_summary = ' '.join(jieba.cut("我喜欢饥饿游戏这本书"))

rouge = Rouge()
scores = rouge.get_scores(generated_summary, reference_summary)
scores

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tassa\AppData\Local\Temp\jieba.cache
Loading model cost 0.450 seconds.
Prefix dict has been built successfully.


[{'rouge-1': {'r': 1.0, 'p': 0.75, 'f': 0.8571428522448981},
  'rouge-2': {'r': 0.6, 'p': 0.375, 'f': 0.4615384568047337},
  'rouge-l': {'r': 1.0, 'p': 0.6666666666666666, 'f': 0.7999999952000001}}]

In [ ]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download("punkt")

def three_sentence_summary(text):
    print(sent_tokenize(text))
    return "\n".join(sent_tokenize(text)[:3])

print(three_sentence_summary("I grew up reading Koontz, and years ago, I stopped,convinced i had outgrown him. Still,when a friend was looking for something suspenseful too read, I suggested Koontz. She found Strangers."))

['I grew up reading Koontz, and years ago, I stopped,convinced i had outgrown him.', 'Still,when a friend was looking for something suspenseful too read, I suggested Koontz.', 'She found Strangers.']
I grew up reading Koontz, and years ago, I stopped,convinced i had outgrown him.
Still,when a friend was looking for something suspenseful too read, I suggested Koontz.
She found Strangers.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tassa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# !pip install nltk
import nltk
# 使用NLTK的英文句子分割
from nltk.tokenize import sent_tokenize
import re
from typing import List
nltk.download("punkt_tab")  # 下载英文句子分割模型
nltk.download("punkt")      # 确保基础分词模型已安装

def chinese_sent_tokenize(text: str) -> List[str]:
    """自定义中文句子分割函数"""
    # 识别中文句子边界的正则表达式
    sentence_endings = r'[。。，，！？…\.\?!?](?:\s|$)'
    sentences = re.split(sentence_endings, text)
    # 过滤空句子并添加句号
    sentences = [s.strip() + '\n' for s in sentences if s.strip()]
    return sentences

def three_sentence_summary(text: str, language: str = "chinese") -> str:
    """提取文本的前三句作为摘要，支持中英文"""
    if language == "chinese":
        sentences = chinese_sent_tokenize(text)
    else:
        sentences = sent_tokenize(text)
    
    # 提取前三句并拼接
    return "\n".join(sentences[:3])

chinese_text = "实在是搞不清楚这本书是太好还是太糟了，也搞不清楚是原作者的文笔就是这么跳跃，还是翻译的过程中丢失了什么隐含含义。 每个字都认识，每个逗号之前还都懂，到了句号经常就无法理解了，更不要提段落和章节了。 整本书读完的感受是，我得等什么时间这本书的内容再从我脑海里自己蹦出来的时候再去读一遍试试。"
print(three_sentence_summary(chinese_text, language="chinese"))

# # 如果需要处理英文文本
# english_text = "This book is amazing. I couldn't put it down. The author's style is brilliant."
# print(three_sentence_summary(english_text, language="english"))    

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\tassa\AppData\Roaming\nltk_data...


实在是搞不清楚这本书是太好还是太糟了，也搞不清楚是原作者的文笔就是这么跳跃，还是翻译的过程中丢失了什么隐含含义

每个字都认识，每个逗号之前还都懂，到了句号经常就无法理解了，更不要提段落和章节了

整本书读完的感受是，我得等什么时间这本书的内容再从我脑海里自己蹦出来的时候再去读一遍试试



[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tassa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text, language="chinese") for text in dataset["review_body"]]
    return metric.get_scores(summaries, dataset["review_title"])

# 来计算验证集上的 ROUGE 分数
import pandas as pd

summaries = [three_sentence_summary(text, language="chinese") for text in books_dataset["validation"]["review_body"]]
test = [' '.join(jieba.cut(summary)) for summary in summaries]
test

['都 是 大道理 方向 ， 各种 整合 ， 设计 变成 了 昂贵 的 资源 而 飞 力量 \n', '非常 好 的 一 本书 。 我 喜欢 投资 ， 希望 认识 一些 对 股票投资 感兴趣 的 朋友 ， 我 的 微信 是 minhua813 \n', '有没有 音频 资料 可以 下载 呢 ？ 我 应该 上 哪里 可以 下载 ？ ？ 急盼 回复 \n', '书本 质量 很 好 ， 纸质 也 够 厚 ， 比当 当好 太多 了 \n', '此书 印刷 粗制滥造 ， 内容 粗浅 教条 ， 是 我 见 过 的 对 读者 最 没有 责任心 的 书籍 之一 ， 作者 一副 高高在上 的 嘴脸 拿 着 这些 东拼西凑 的 内容 仿佛 在 嘲弄 每 一位 读者 ： 你们 这群 傻 逼 ， 挣 你们 的 钱 太 容易 了 \n', '课文 中 题目 的 答案 在 辅助 用 书上 ， 所以 买书 的 同学 们 ， 要 两本书 一起 买才行 \n', '语句 不通 ， 错漏 百出 。 内容 也 一般 ， 可读性 差 \n', '話 の 筋 は 面白 い か も 知 れ な い け れ ど 、 そ も そ も こ の 内容 を こ ん な に 長 く す る 必要 は あ っ た の だ ろ う か ？ や は り 、 こ の 作家 さ ん は 、 短編 の 方 が い い か も \n', '本来 就是 买来 当 参考书 考研 用 的 ， 暂时 没 仔细 ， 粗略 翻 了 下 。 内容 比较 适合 ， 比较 有 纲领性 \n \n 总体性 很强 ， 以至于 有 不少 部分 和 其他 书有 重复 \n', '收到 后 第一 感觉 是 高大 上 ， 首先 装帧 设计 非常 精美 ， 很 有 特色 的 翻页 设计 ， 其次 是 内容 精彩 ， 中英对照 阅读 方便 ， 提高 了 读者 的 阅读 兴趣 ， 同时 让 读者 足不出户 就 丰富 了 大脑 ， 读后 让 人 开拓 了 视野 、 满足 了 感官 神经 、 补给 了 营养 ， 实属 一本 难得 的 好书 ， 强烈推荐 \n', '三 本书 里 最贵 的 ， 也 是 质量 最差 的 ， 封面 的 硬纸 壳 已经 开裂 了 ， 总之 质量 比 19 块 买 的 1984 差太多 。 书本 纸张 质量 还 不错 \n', '很 好 的 一 本书

[{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
 {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  '